In [1]:
from library.matrix import Matrix, zeros
from library.linear_equations import forward_propagation, backward_propagation, LU_Decomposition, make_diag_dominant, gauss_seidel

In [2]:
precision = 3

A = Matrix(
    [
        [1, 0, 2],
        [3, 0, 2],
        [-1, 1, 2]
    ], "A", precision
)

# B = Matrix([[20], [15], [92], [51], [15]], "B", precision)

# A, B = make_diag_dominant(A, B)

In [3]:
L, U = LU_Decomposition(A)
print(L)
print(U)

ZeroDivisionError: float division by zero